In [4]:
import numpy as np
import h5py
from glob import glob
from data_understanding import save_meta_data,read_meta_data

dict_keys(['building', 'vegetation', 'ground', 'undefined'])

In [12]:
id_class = {'building':0, 'vegetation':1, 'ground':2, 'undefined':3}

In [7]:
metadata_json = glob("T**")

In [18]:
xyz_rgb = np.load(metadata[class_][inst_ids_keys[0]]["path_all_data"],allow_pickle=True)

NameError: name 'inst_ids_keys' is not defined

In [23]:
metadata[class_][inst_ids_keys[0]]["path_all_data"]

TypeError: 'dict_keys' object is not subscriptable

In [ ]:
uint32_

In [30]:
8**3

512

In [40]:
x = np.zeros(xyz_rgb.shape[0],dtype="uint16")
x.fill(255)

In [46]:
xyz_rgb = np.load(metadata[class_][inst_ids_keys[0]]["path_all_data"],allow_pickle=True)
vec_size = xyz_rgb.shape[0]
ins_id = np.zeros(vec_size,dtype="uint16")
ins_id.fill(int(inst_ids_keys[0])) 
class_id= np.zeros(vec_size,dtype="uint16")
class_id.fill(id_class['building'])

array([0, 0, 0, ..., 0, 0, 0], dtype=uint16)

In [70]:
def append_xyz_rgb_class_ins_id(new_data,old_data):
    xyz = np.append(old_data[0],new_data[0])
    rgb = np.append(old_data[1],new_data[1])
    class_id = np.append(old_data[2],new_data[2])
    ins_id = np.append(old_data[3],new_data[3])
    return xyz,rgb,class_id,ins_id


In [85]:
def read_xyz_rgb_class_ins_id(class_,inst_ids_keys,index,metadata,path,new_data=None):
    if class_ == "building":
        xyz_rgb = np.load(metadata[class_][inst_ids_keys[index]][path],allow_pickle=True)
    else: 
        xyz_rgb = np.loadtxt(f"Dataset/{metadata[class_][inst_ids_keys[index]][path]}")
    vec_size = xyz_rgb.shape[0]
    ins_id = np.zeros(vec_size,dtype="uint16")
    ins_id.fill(int(inst_ids_keys[index])) 
    class_id= np.zeros(vec_size,dtype="uint16")
    class_id.fill(id_class[class_])
    if new_data:
        return append_xyz_rgb_class_ins_id(new_data,[xyz_rgb[:,:3],xyz_rgb[:,3:].astype("int"),class_id,ins_id])
    return [xyz_rgb[:,:3],xyz_rgb[:,3:].astype("int"),class_id,ins_id]

def combine_all_class_to_area(building,vegetation,ground,undefined):
    pass

In [86]:
for area in metadata_json:
    metadata = read_meta_data(area)
    for class_ in metadata.keys():
        print(class_)
        if class_ == "building" :
            inst_ids_keys = list(metadata[class_].keys())
            buff_data =  read_xyz_rgb_class_ins_id(class_="building",path="path_all_data",inst_ids_keys=inst_ids_keys,index=0,metadata=metadata)
            for idx in range(1,len(inst_ids_keys),1):
                buff_data = read_xyz_rgb_class_ins_id(class_="building",path="path_all_data",inst_ids_keys=inst_ids_keys,index=0,metadata=metadata,new_data=buff_data)
            building=buff_data
        if class_ == "vegetation" :
            inst_ids_keys = list(metadata[class_].keys())
            buff_data =  read_xyz_rgb_class_ins_id(class_="vegetation",path="path",inst_ids_keys=inst_ids_keys,index=0,metadata=metadata)
            try:
                for idx in range(1,len(inst_ids_keys),1):
                    buff_data = read_xyz_rgb_class_ins_id(class_="vegetation",path="path",inst_ids_keys=inst_ids_keys,index=0,metadata=metadata,new_data=buff_data)
            except:
                pass
            vegetation=buff_data
        if class_ == "ground" :
            inst_ids_keys = list(metadata[class_].keys())
            buff_data =  read_xyz_rgb_class_ins_id(class_="ground",path="path",inst_ids_keys=inst_ids_keys,index=0,metadata=metadata)
            try:
                for idx in range(1,len(inst_ids_keys),1):
                    buff_data = read_xyz_rgb_class_ins_id(class_="ground",path="path",inst_ids_keys=inst_ids_keys,index=0,metadata=metadata,new_data=buff_data)
            except:
                pass
            ground=buff_data
        if class_ == "undefined":
            inst_ids_keys = list(metadata[class_].keys())
            buff_data =  read_xyz_rgb_class_ins_id(class_="undefined",path="path",inst_ids_keys=inst_ids_keys,index=0,metadata=metadata)
            try:
                for idx in range(1,len(inst_ids_keys),1):
                    buff_data = read_xyz_rgb_class_ins_id(class_="undefined",path="path",inst_ids_keys=inst_ids_keys,index=0,metadata=metadata,new_data=buff_data)
            except:
                pass
            undefined=buff_data
        break

vegetation


In [80]:
type(buff_data[0])

numpy.ndarray

In [13]:
metadata["building"]

{'000': {'total_point': 226122,
  'path': ['T_316500_234000_SW_T_316500_233500_NW/building_01.txt',
   'T_316500_234000_SW_T_316500_233500_NW/building_01_door_01.txt',
   'T_316500_234000_SW_T_316500_233500_NW/building_01_door_02.txt',
   'T_316500_234000_SW_T_316500_233500_NW/building_01_door_03.txt',
   'T_316500_234000_SW_T_316500_233500_NW/building_01_roof_01.txt',
   'T_316500_234000_SW_T_316500_233500_NW/building_01_roof_01_window_01.txt',
   'T_316500_234000_SW_T_316500_233500_NW/building_01_roof_01_window_02.txt',
   'T_316500_234000_SW_T_316500_233500_NW/building_01_roof_01_window_03.txt',
   'T_316500_234000_SW_T_316500_233500_NW/building_01_roof_01_window_04.txt',
   'T_316500_234000_SW_T_316500_233500_NW/building_01_roof_01_window_05.txt',
   'T_316500_234000_SW_T_316500_233500_NW/building_01_roof_01_window_06.txt',
   'T_316500_234000_SW_T_316500_233500_NW/building_01_roof_01_window_07.txt',
   'T_316500_234000_SW_T_316500_233500_NW/building_01_roof_01_window_08.txt',
   '